In [1]:
from large_gcs.algorithms.search_algorithm import AlgMetrics, SearchNode
from large_gcs.domination_checkers.reaches_new_containment import ReachesNewContainment
from large_gcs.graph.incremental_contact_graph import IncrementalContactGraph
from large_gcs.graph_generators.contact_graph_generator import ContactGraphGeneratorParams
from large_gcs.geometry.polyhedron import Polyhedron

import numpy as np
import logging
from pydrake.all import (MathematicalProgram, HPolyhedron)

logging.basicConfig(level=logging.WARN)
logging.getLogger("large_gcs").setLevel(logging.DEBUG)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
from large_gcs.domination_checkers.reaches_cheaper_containment import ReachesCheaperContainment


Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-05


/home/shaoyuan/Documents/Software/pypolycontain/pypolycontain/conversions.py:19: UserWarning: WARNING: You don't have CDD package installed. Unable to visualize polytopes. You may still visualize zonotopes.
  warnings.warn("WARNING: You don't have CDD package installed. Unable to visualize polytopes. You may still visualize zonotopes.")
/home/shaoyuan/Documents/Software/pypolycontain/pypolycontain/containment.py:16: UserWarning: You don't have CDD package installed. Unable to run cone ray generation.
  warnings.warn("You don't have CDD package installed. Unable to run cone ray generation.")


In [2]:

from large_gcs.domination_checkers.ah_containment_last_pos import ReachesCheaperLastPosContainment
from large_gcs.domination_checkers.sampling_containment_domination_checker import ReachesCheaperLastPosSamplingContainment
from large_gcs.domination_checkers.sampling_containment_double_domination_checker import ReachesCheaperLastPosSamplingContainmentDouble


graph_file = ContactGraphGeneratorParams.inc_graph_file_path_from_name(
    "cg_simple_5"
)
cg = IncrementalContactGraph.load_from_file(
    graph_file,
    should_incl_simul_mode_switches=False,
    should_add_const_edge_cost=True,
    should_add_gcs=True,
    should_use_l1_norm_vertex_cost=True,
)
dom_ns = ReachesCheaperLastPosContainment(graph=cg, construct_path_from_nullspaces=True)
dom_ns.set_alg_metrics(AlgMetrics())
dom_fs = ReachesCheaperLastPosContainment(graph=cg, construct_path_from_nullspaces=False)
dom_fs.set_alg_metrics(AlgMetrics())
dom_double = ReachesCheaperLastPosSamplingContainmentDouble(graph=cg, construct_path_from_nullspaces=False, num_samples_per_vertex=1)
dom_double.set_alg_metrics(AlgMetrics())

INFO:large_gcs.graph.contact_graph:Creating vertex costs...
100%|██████████| 2/2 [00:00<00:00, 44384.17it/s]
INFO:large_gcs.graph.graph:Adding 2 vertices to graph...
100%|██████████| 2/2 [00:00<00:00, 21454.24it/s]
INFO:large_gcs.graph.contact_graph:Generating contact pair modes for 3 body pairs...
100%|██████████| 3/3 [00:00<00:00, 181.42it/s]
INFO:large_gcs.graph.incremental_contact_graph:Calculating adjacent contact pair modes (62)
100%|██████████| 62/62 [00:00<00:00, 9483.84it/s]


In [3]:
cand_path =['source', "('NC|obs0_f1-obj0_f1', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'NC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f2-rob0_f0')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f3-rob0_f1')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'NC|obj0_f3-rob0_f1')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')", "('NC|obs0_f2-obj0_f2', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')"]
alt_path  =['source', "('NC|obs0_f1-obj0_f1', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'NC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f2-rob0_f0')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'NC|obj0_f3-rob0_f1')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')", "('NC|obs0_f2-obj0_f2', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')"]

cg.add_vertex_path_to_graph(cand_path)
cg.add_vertex_path_to_graph(alt_path)


In [4]:
len(cand_path)

9

In [5]:
cand_node = SearchNode.from_vertex_path(cand_path)
alt_node = SearchNode.from_vertex_path(alt_path)
dom_double.is_dominated(cand_node, [alt_node])
# finishes in 6.7s

DEBUG:large_gcs.domination_checkers.sampling_domination_checker:Adding samples for ('NC|obs0_f2-obj0_f2', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')
DEBUG:large_gcs.geometry.convex_set:Sampled 1 points from convex set
DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:create_path_AH_polytope_from_nullspace_sets
DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:nullspace_polyhedron_and_transformation_from_HPoly_and_T: Shape of input h_poly: (314, 78)
DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:create_path_AH_polytope_from_full_sets
DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:nullspace_polyhedron_and_transformation_from_HPoly_and_T: Shape of input h_poly: (398, 120)
DEBUG:large_gcs.domination_checkers.sampling_containment_double_domination_checker:Checking domination of candidate node terminating at vertex ('NC|obs0_f2-obj0_f2', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')
 via path: ['source', "('

False

In [8]:
cand_node = SearchNode.from_vertex_path(cand_path)
alt_node = SearchNode.from_vertex_path(alt_path)
dom_fs.is_dominated(cand_node, [alt_node])
# finishes in 3.4s

DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:Checking domination of candidate node terminating at vertex ('NC|obs0_f2-obj0_f2', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')
 via path: ['source', "('NC|obs0_f1-obj0_f1', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'NC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f2-rob0_f0')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f3-rob0_f1')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'NC|obj0_f3-rob0_f1')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')", "('NC|obs0_f2-obj0_f2', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')"]
DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:create_path_AH_polytope_from_full_sets
DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:nullspace_polyhedron_and_transformation_fr

False

In [9]:
cand_node = SearchNode.from_vertex_path(cand_path)
alt_node = SearchNode.from_vertex_path(alt_path)
dom_ns.is_dominated(cand_node, [alt_node])

DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:Checking domination of candidate node terminating at vertex ('NC|obs0_f2-obj0_f2', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')
 via path: ['source', "('NC|obs0_f1-obj0_f1', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'NC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f1-rob0_f3')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f2-rob0_f0')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'IC|obj0_f3-rob0_f1')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f1-rob0_f1', 'NC|obj0_f3-rob0_f1')", "('NC|obs0_f1-obj0_f1', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')", "('NC|obs0_f2-obj0_f2', 'NC|obs0_f0-rob0_f0', 'NC|obj0_f3-rob0_f1')"]
DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:create_path_AH_polytope_from_nullspace_sets
DEBUG:large_gcs.domination_checkers.ah_containment_domination_checker:nullspace_polyhedron_and_transformati